## Enhancement 5: Choose a new dataset from the list below. Search the Internet and download your chosen dataset (many of them could be available on kaggle). Adapt your model to your dataset. Train your model and record your results.

   * cancer_dataset          - Breast cancer dataset.
   * crab_dataset            - Crab gender dataset.
   * glass_dataset           - Glass chemical dataset.
   * iris_dataset            - Iris flower dataset.
   * ovarian_dataset         - Ovarian cancer dataset.
   * thyroid_dataset         - Thyroid function dataset.

#### Imports

In [19]:
import warnings
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#### Load Dataset

In [20]:
from sklearn.datasets import load_breast_cancer
cancer_dataset = load_breast_cancer()

In [21]:
type(cancer_dataset)

sklearn.utils.Bunch

In [22]:
df = pd.DataFrame(data=cancer_dataset.data, columns=cancer_dataset.feature_names)
df['target'] = cancer_dataset.target

In [23]:
# make sure it looks good
df.head()
# df.tail()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [24]:
# how many features?
# WOW!
len(df.columns) -1

30

In [25]:
# 0 means malignant
# 1 means benign
# @$@ switch these
df['target'].tail(5)

564    0
565    0
566    0
567    0
568    1
Name: target, dtype: int32

In [26]:
# normalize data so that each feature has a mean of 0 and a std of 1
labels = df['target']
# @$@ there is a better way
# if vals too big. -> maybe each col normailzed seperatly
df = (df - df.mean()) / df.std()
df['target'] = labels

In [27]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,1.096100,-2.071512,1.268817,0.983510,1.567087,3.280628,2.650542,2.530249,2.215566,2.253764,...,-1.358098,2.301575,1.999478,1.306537,2.614365,2.107672,2.294058,2.748204,1.935312,0
1,1.828212,-0.353322,1.684473,1.907030,-0.826235,-0.486643,-0.023825,0.547662,0.001391,-0.867889,...,-0.368879,1.533776,1.888827,-0.375282,-0.430066,-0.146620,1.086129,-0.243675,0.280943,0
2,1.578499,0.455786,1.565126,1.557513,0.941382,1.052000,1.362280,2.035440,0.938859,-0.397658,...,-0.023953,1.346291,1.455004,0.526944,1.081980,0.854222,1.953282,1.151242,0.201214,0
3,-0.768233,0.253509,-0.592166,-0.763792,3.280667,3.399917,1.914213,1.450431,2.864862,4.906602,...,0.133866,-0.249720,-0.549538,3.391291,3.889975,1.987839,2.173873,6.040726,4.930672,0
4,1.748758,-1.150804,1.775011,1.824624,0.280125,0.538866,1.369806,1.427237,-0.009552,-0.561956,...,-1.465481,1.337363,1.219651,0.220362,-0.313119,0.612640,0.728618,-0.867590,-0.396751,0


In [28]:
# sumamry statistics of the data
df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
count,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,...,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,569.000000
mean,-3.159355e-15,-6.549730e-15,-6.993039e-16,-8.553985e-16,6.043984e-15,-1.111394e-15,-2.997017e-16,1.023981e-15,-1.860648e-15,-1.461046e-15,...,1.735772e-15,-1.211294e-15,6.243785e-16,-5.094929e-15,-2.097912e-15,6.368661e-16,-1.998011e-16,-2.422589e-15,2.497514e-15,0.627417
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.483918
min,-2.027864e+00,-2.227289e+00,-1.982759e+00,-1.453164e+00,-3.109349e+00,-1.608721e+00,-1.113893e+00,-1.260710e+00,-2.741705e+00,-1.818265e+00,...,-2.222039e+00,-1.691872e+00,-1.221348e+00,-2.680337e+00,-1.442609e+00,-1.304683e+00,-1.743529e+00,-2.159060e+00,-1.600431e+00,0.000000
25%,-6.887793e-01,-7.253249e-01,-6.913472e-01,-6.666089e-01,-7.103378e-01,-7.464292e-01,-7.430941e-01,-7.372951e-01,-7.026215e-01,-7.220040e-01,...,-7.479711e-01,-6.889721e-01,-6.415713e-01,-6.906227e-01,-6.804845e-01,-7.558491e-01,-7.557349e-01,-6.412994e-01,-6.913035e-01,0.000000
50%,-2.148925e-01,-1.045442e-01,-2.357726e-01,-2.949274e-01,-3.486040e-02,-2.217454e-01,-3.419391e-01,-3.973715e-01,-7.156354e-02,-1.781226e-01,...,-4.347738e-02,-2.857288e-01,-3.408813e-01,-4.680159e-02,-2.692639e-01,-2.180402e-01,-2.232725e-01,-1.272975e-01,-2.162538e-01,1.000000
75%,4.689800e-01,5.836621e-01,4.992377e-01,3.631877e-01,6.356397e-01,4.934227e-01,5.255994e-01,6.463664e-01,5.303125e-01,4.705693e-01,...,6.577623e-01,5.398040e-01,3.572747e-01,5.970195e-01,5.391944e-01,5.306742e-01,7.118836e-01,4.497425e-01,4.503661e-01,1.000000
max,3.967796e+00,4.647799e+00,3.972634e+00,5.245913e+00,4.766717e+00,4.564409e+00,4.239858e+00,3.924477e+00,4.480808e+00,4.906602e+00,...,3.882489e+00,4.283568e+00,5.924959e+00,3.951897e+00,5.108382e+00,4.696536e+00,2.683516e+00,6.040726e+00,6.840837e+00,1.000000


## Load this dataset for training a neural network

In [29]:
# The dataset class
# inherit the Dataset class and create a new class CancerDataset
class CancerDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.features = []
        self.labels = []
        
        # Iterate through rows of the DataFrame
        for _, row in df.iterrows():
            # Append features for each row except the 'target' column
            self.features.append(row.drop('target').tolist())
            # Append label for each row
            self.labels.append(row['target'])
    
    # set length to reaturn number of rows
    def __len__(self):
        return len(self.df)
    
    # get a sample in the form of a dictionary (features and labels) given its index
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        features = self.features[idx]
        features = torch.FloatTensor(features)

        labels = torch.tensor(self.labels[idx], dtype = torch.long)

        return {'labels': labels, 'features': features}

# instanciate a CancerDataset object based off of data_df
cancer_dataset = CancerDataset(df)
# perform a 80, 10, 10 split using the cancer_dataset object
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(cancer_dataset, [0.8, 0.1, 0.1])

# The dataloader
train_dataloader = DataLoader(
    # dataset input
    train_dataset,
    # batches of 4 during training
    batch_size = 4,
    # don't learn from the order of the dataset!
    shuffle = True,
    # multy processing. (0 = 1 process to load data)
    # how many processes can you do? 
    num_workers = 0
)

#want results without any randomness, therfore shuffle is False for these
val_dataloader = DataLoader(val_dataset, batch_size = 4, shuffle = False, num_workers = 0)
test_dataloader = DataLoader(test_dataset, batch_size = 4, shuffle = False, num_workers = 0)

In [30]:
# peak into the dataset
for i in cancer_dataset:
    print(i)
    break

{'labels': tensor(0), 'features': tensor([ 1.0961, -2.0715,  1.2688,  0.9835,  1.5671,  3.2806,  2.6505,  2.5302,
         2.2156,  2.2538,  2.4875, -0.5648,  2.8305,  2.4854, -0.2138,  1.3157,
         0.7234,  0.6602,  1.1477,  0.9063,  1.8850, -1.3581,  2.3016,  1.9995,
         1.3065,  2.6144,  2.1077,  2.2941,  2.7482,  1.9353])}


## Neural Network

In [31]:
# change the device to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
#defines new class named CancerModel, inherits from torch.blah
class CancerModel(torch.nn.Module):

    def __init__(self):
        # initialize in the same way the parent class does first
        super(CancerModel, self).__init__()
        
        # create linear layer. 11 inputs 200 outputs?
        self.linear1 = torch.nn.Linear(30, 210)                
        # creates activation function (What does this do and how does it work @$@)
        self.activation = torch.nn.ReLU()
        
        # a second linear layer. Takes in 200 and outputs 6 (one corresponding to each feature)
        self.linear2 = torch.nn.Linear(210, 2)              
         
        # sigmoid activation for binary
        self.softmax = torch.nn.Softmax()
    
    # defines forward pass of nn
    # sends input through the tunnel: lin1 -> act -> line2 -> softmax 
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.softmax(x)
        return x

# earlier we set device to cpu or gpu
# create this CancerModel object, then move to cpu or gpu
cancermodel = CancerModel().to(device)

In [33]:
# Define and the loss function and optimizer
# we are trying binary cross entropy loss
# what is BCE using as inputs
    # 
criterion = nn.CrossEntropyLoss().to(device)

# an optimizer @$@ what does it do? 
# specify the learning rate
optimizer = AdamW(cancermodel.parameters(), lr = 1e-3)

In [34]:
# Lets define the training steps
def accuracy(preds, labels):
    preds = torch.argmax(preds, dim=1).flatten()
    return torch.sum(preds == labels) / len(labels)

def train(model, data_loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    # activate training mode
    model.train()
    
    # iterate over batches
    for d in tqdm(data_loader):
        inputs = d['features'].to(device)
        #labels = d['labels'].unsqueeze(1).to(device)  # Adjusting the shape of the target tensor
        labels = d['labels'].to(device)  # Adjusting the shape of the target tensor

        #labels = labels.float()  # Cast labels to float32
        
        # obtaining predictions
        outputs = cancermodel(inputs)
        preds = (outputs > 0.5).float()  # Convert logits to binary predictions
        # calulate loss and accuracy by comparing predictions to labels
        loss = criterion(outputs, labels)
        acc = accuracy(preds, labels)  # Use binary accuracy function
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(data_loader), epoch_acc / len(data_loader)


# Lets define the testing steps
def evaluate(model, data_loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0
    
    # eval mode activate!
    model.eval()
    
    # iterate over batches (later we will average the huge sum)
    with torch.no_grad():
        for d in data_loader:
            # extract labels and features from batch and send to cpu/gpu
            inputs = d['features'].to(device)
            labels = d['labels'].to(device)  # Adjusting the shape of the target tensor
            # labels = labels.float()  # Convert labels to float32
            
            # use model to get predictions
            outputs = cancermodel(inputs)
            
            #  get loss and accuracy
            loss = criterion(outputs, labels)
            acc = accuracy(outputs, labels)
            
            # run a big sum!
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            precision = precision_score(labels.cpu(), outputs.argmax(dim=1).cpu() > 0.5, average='weighted')
            recall = recall_score(labels.cpu(), outputs.argmax(dim=1).cpu() > 0.5, average='weighted')
            f1 = f1_score(labels.cpu(), outputs.argmax(dim=1).cpu() > 0.5, average='weighted')
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    # average big sums by # of epochs
    num_batches = len(data_loader)
    return (epoch_loss / num_batches, 
            epoch_acc / num_batches, 
            epoch_precision / num_batches, 
            epoch_recall / num_batches,
            epoch_f1 / num_batches)


In [36]:
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Let's train our model
for epoch in range(50):
    train_loss, train_acc = train(cancermodel, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc, valid_precision, valid_recall, valid_f1 = evaluate(cancermodel, val_dataloader, criterion)

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% | Val. Precision: {valid_precision:.3f} | Val. Recall: {valid_recall:.3f} | Val. F1-score: {valid_f1:.3f} |')


  0%|          | 0/114 [00:00<?, ?it/s]

C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


| Epoch: 01 | Train Loss: 0.468 | Train Acc: 85.53% | Val. Loss: 0.401 | Val. Acc: 88.33% | Val. Precision: 0.917 | Val. Recall: 0.883 | Val. F1-score: 0.893 |


C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\t

  0%|          | 0/114 [00:00<?, ?it/s]

...

| Epoch: 12 | Train Loss: 0.322 | Train Acc: 99.34% | Val. Loss: 0.389 | Val. Acc: 90.00% | Val. Precision: 0.917 | Val. Recall: 0.900 | Val. F1-score: 0.902 |


C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\t

  0%|          | 0/114 [00:00<?, ?it/s]

C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\t

KeyboardInterrupt: 

In [37]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

test_loss, test_acc, test_prec, test_rec, test_f1 = evaluate(cancermodel, test_dataloader, criterion)


print(f'| Test. Loss: {test_loss:.3f} | Test. Acc: {test_acc*100:.2f}% | Test. Precision: {test_prec:.3f} | Test. Recall: {test_rec:.3f} | Test. F1-score: {test_f1:.3f} |')

| Test. Loss: 0.339 | Test. Acc: 98.21% | Test. Precision: 0.988 | Test. Recall: 0.982 | Test. F1-score: 0.981 |


C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\dansc\anaconda3\lib\site-packages\t

In [ ]:
warnings.resetwarnings()